In [1]:
#Omero
from myconfig import OMEROUSER, OMEROPASS, OMEROPORT, OMEROHOST
import ezomero
from omero.model import OriginalFileI, PlateI, ScreenPlateLinkI, ScreenI
from omero.rtypes import rint, rlong, rstring, robject, unwrap



#Other
import h5py
import pandas as pd
import skimage.io
import os
import pathlib
import pickle
import tempfile
import skimage
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime
import warnings
import time
import glob
import PIL
import re
import cv2

In [2]:
#Parameters

# OMERO IDs
plate_id = 765 #Insert corresponding project ID
dataset_id = 7801

In [3]:
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)
dataset = conn.getObject("Dataset", dataset_id)

In [4]:
# Create new plate
results_screen_name = "Results_of_IDR0028_Plate1A"
results_screen_id = ezomero.post_screen(conn, results_screen_name, description="These are the results of the CP analysis.") 

# Create new plate
plate = conn.getObject("Plate", plate_id)
plate_name = plate.name
results_plate = PlateI()
results_plate.name = rstring(plate_name)
results_plate = conn.getUpdateService().saveAndReturnObject(results_plate)
results_plate_id = results_plate.getId()

# Links new Plate with new Screen
link = ScreenPlateLinkI()
link.setParent(ScreenI(results_screen_id, False))
link.setChild(PlateI(results_plate_id, False))
link_update_service = conn.getUpdateService()
link_update_service.saveObject(link)

In [5]:
rows_columns = []
image_ids = []

for image in dataset.listChildren():
    image_ids.append(image.id)
    rows_columns.append((int(image.name[:3]), int(image.name[3:6])))

#print(len(image_ids))

6716


In [6]:
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)
for (row, column) in set(rows_columns):
    images_select = []
    print(f"Dataset to Plate: {row, column}")
    for i, rc in enumerate(rows_columns): 
        if rc == (row, column):
            images_select.append(image_ids[i])
    
    update_service = conn.getUpdateService()

    well = omero.model.WellI()
    well.plate = omero.model.PlateI(results_plate_id, False)
    well.column = rint(column)
    well.row = rint(row)
    
    try:
        for image in images_select:
            ws = omero.model.WellSampleI()
            ws.image = omero.model.ImageI(image, False)
            ws.well = well
            well.addWellSample(ws)
        update_service.saveObject(well)
    except Exception:
        print("error")   

Dataset to Plate: (12, 4)
Dataset to Plate: (6, 18)
Dataset to Plate: (7, 17)
Dataset to Plate: (4, 9)
Dataset to Plate: (3, 13)
Dataset to Plate: (5, 10)
Dataset to Plate: (8, 9)
Dataset to Plate: (10, 6)
Dataset to Plate: (9, 8)
Dataset to Plate: (3, 22)
Dataset to Plate: (5, 19)
Dataset to Plate: (8, 18)
Dataset to Plate: (11, 5)
Dataset to Plate: (9, 17)
Dataset to Plate: (11, 14)
Dataset to Plate: (14, 22)
Dataset to Plate: (13, 8)
Dataset to Plate: (13, 17)
Dataset to Plate: (6, 11)
Dataset to Plate: (7, 10)
Dataset to Plate: (6, 20)
Dataset to Plate: (7, 19)
Dataset to Plate: (3, 6)
Dataset to Plate: (5, 3)
Dataset to Plate: (12, 18)
Dataset to Plate: (3, 15)
Dataset to Plate: (5, 12)
Dataset to Plate: (8, 11)
Dataset to Plate: (14, 15)
Dataset to Plate: (9, 10)
Dataset to Plate: (11, 7)
Dataset to Plate: (5, 21)
Dataset to Plate: (9, 19)
Dataset to Plate: (11, 16)
Dataset to Plate: (13, 10)
Dataset to Plate: (6, 4)
Dataset to Plate: (7, 3)
Dataset to Plate: (10, 20)
Dataset to 

In [9]:
# remove from Dataset
dataset_id = 7801
remove_from = conn.getObject("Dataset", dataset_id)
for image_id in image_ids:
    if remove_from is not None:
        image = conn.getObject("Image", image_id)
        links = list(image.getParentLinks(remove_from.id))
        link_ids = [l.id for l in links]
        conn.deleteObjects('DatasetImageLink', link_ids)
